# Test Processing of a single cycling data

The cycling data will include both RPTs and information during cycling. First, split up these two pieces. Then, for each piece, parse the data.

Compile information on UMBLFEB2022 formation cells

1/17/2023

In [81]:
import os, sys
from matplotlib import pyplot as plt

if os.path.basename(os.getcwd()) == 'notebooks-2023':
    os.chdir('../')
    sys.path.insert(0, 'src/')
    
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib ipympl

from scipy import signal, ndimage
import src.plotter as plotter
import src.vas as vas
import src.parsers as parsers

In [2]:
plotter.initialize(plt)

In [3]:
# Initialize the helper class
vh = vas.VasHelper()

Initializing Voltaiq Analytic Studio Helper...
Initializing test records...
Initializing devices...
Done.


In [29]:
test_list, _ = vh.get_test_names('UMBL2022FEB_CELL152028')

cyc_test = []
for test_name in test_list:
    if 'CYC' in test_name:
        cyc_test.append(test_name)
        
assert not len(cyc_test) == 0, 'No cycling test found.'
assert not len(cyc_test) > 1, 'More than one cycling test found.'

cyc_test = cyc_test[0]
cyc_test

'UMBL2022FEB_CELL152028_CYC_1C1CR1_P45C_5P0PSI_20220804_R1'

In [30]:
df = vh.get_cycler_data(cyc_test)
df

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,step_time_s,current_a,voltage_v,step_index,discharge_energy_wh,test_time_s,cycle_index,datetime
0,0.000000,0.000000,0.000000,0.054043,0.0,3.431613,1,0.000000,5.404310e-02,1,2022-08-04 10:33:31-04:00
1,0.000000,0.000000,0.000000,10.065207,0.0,3.431613,1,0.000000,1.006521e+01,1,2022-08-04 10:33:41-04:00
2,0.000000,0.000000,0.000000,20.073750,0.0,3.431613,1,0.000000,2.007375e+01,1,2022-08-04 10:33:51-04:00
3,0.000000,0.000000,0.000000,30.084908,0.0,3.431776,1,0.000000,3.008491e+01,1,2022-08-04 10:34:01-04:00
4,0.000000,0.000000,0.000000,40.098191,0.0,3.431613,1,0.000000,4.009819e+01,1,2022-08-04 10:34:11-04:00
...,...,...,...,...,...,...,...,...,...,...,...
1659160,1.250625,5.238502,1.821491,910715.425771,0.0,3.392303,31,5.876043,1.504838e+07,1578,2023-01-31 08:55:38-05:00
1659161,1.250625,5.238502,1.821491,910725.427595,0.0,3.392303,31,5.876043,1.504839e+07,1578,2023-01-31 08:55:48-05:00
1659162,1.250625,5.238502,1.821491,910735.433084,0.0,3.392303,31,5.876043,1.504840e+07,1578,2023-01-31 08:55:58-05:00
1659163,1.250625,5.238502,1.821491,910745.445456,0.0,3.392465,31,5.876043,1.504841e+07,1578,2023-01-31 08:56:08-05:00


In [10]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(8, nrows * 3), sharex=True)

i = 0
for yvar, ax in zip(plot_vars, axs):
    ax.plot(df[xvar], df[yvar])
    ax.set_ylabel(yvar)
    if i < len(axs):
        ax.set_xticks([])
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
df_agg

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,step_time_s,current_a,voltage_v,step_index,discharge_energy_wh,test_time_s,datetime
cycle_index,,,,,,,,,,
1,0.000000,0.000000,0.192217,9826.524841,0.000000,3.431938,13,0.612502,1.970852e+04,2022-08-04 16:02:00-04:00
2,0.081963,0.268955,0.006945,1800.011709,2.500492,3.381094,19,0.020153,2.294463e+04,2022-08-04 16:55:57-04:00
3,0.081969,0.281453,0.006950,1800.005453,2.500492,3.489930,19,0.021867,2.618077e+04,2022-08-04 17:49:54-04:00
4,0.081970,0.287464,0.006947,1800.016697,2.500673,3.537363,19,0.022767,2.941688e+04,2022-08-04 18:43:52-04:00
5,0.081971,0.289103,0.006954,1800.017690,2.500492,3.565790,19,0.023104,3.265302e+04,2022-08-04 19:37:49-04:00
...,...,...,...,...,...,...,...,...,...,...
1574,1.257079,5.264991,1.254994,5075.575957,2.500854,4.200124,9,4.117342,1.410069e+07,2023-01-20 09:40:54-05:00
1575,1.255640,5.259053,1.253413,5082.838992,2.500854,4.200124,9,4.112072,1.410918e+07,2023-01-20 12:02:21-05:00
1576,1.254240,5.253376,1.252168,5082.480913,2.500673,4.200124,9,4.108112,1.411766e+07,2023-01-20 14:23:43-05:00


,cycle_index,tot_charge_capacity_ah,tot_charge_energy_wh,tot_discharge_capacity_ah,tot_discharge_energy_wh
0,1,NaN,NaN,NaN,NaN
1,2,0.000000,0.000000,0.192217,0.612502
2,3,0.081963,0.268955,0.199162,0.632655
3,4,0.163932,0.550408,0.206113,0.654522
4,5,0.245902,0.837872,0.213059,0.677289
...,...,...,...,...,...
1573,1574,2244.145393,9039.157962,2242.324649,7702.637481
1574,1575,2245.402472,9044.422953,2243.579642,7706.754823
1575,1576,2246.658112,9049.682006,2244.833056,7710.866895
1576,1577,2247.912352,9054.935383,2246.085223,7714.975007


In [116]:
RPT_STEP_INDEX_START         = 12
RPT_STEP_INDEX_END           = 31
RPT_STEP_INDEX_C20_CHARGE    = 26
RPT_STEP_INDEX_C20_DISCHARGE = 23

df_cyc = df[df['step_index'] < RPT_STEP_INDEX_START]
df_rpt = df[(df['step_index'] >= RPT_STEP_INDEX_START) & (df['step_index'] <= RPT_STEP_INDEX_END)]

In [123]:
df_agg.loc[df_cyc['cycle_index'].unique()]

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,step_time_s,current_a,voltage_v,step_index,discharge_energy_wh,test_time_s,datetime
cycle_index,,,,,,,,,,
1,0.000000,0.000000,0.192217,9826.524841,0.000000,3.431938,13,0.612502,1.970852e+04,2022-08-04 16:02:00-04:00
35,0.000000,0.000000,2.413580,25420.176096,0.000000,4.185342,31,8.778805,3.452497e+05,2022-08-08 10:28:18-04:00
36,2.407661,9.304103,2.358444,3395.707001,2.500673,4.200124,9,8.455596,3.542531e+05,2022-08-08 12:58:22-04:00
37,2.362159,9.137254,2.356331,3392.727016,2.500673,4.200124,9,8.446867,3.631546e+05,2022-08-08 15:26:44-04:00
38,2.357555,9.121293,2.352764,3387.582370,2.500673,4.200124,9,8.431586,3.720449e+05,2022-08-08 17:54:55-04:00
...,...,...,...,...,...,...,...,...,...,...
1574,1.257079,5.264991,1.254994,5075.575957,2.500854,4.200124,9,4.117342,1.410069e+07,2023-01-20 09:40:54-05:00
1575,1.255640,5.259053,1.253413,5082.838992,2.500854,4.200124,9,4.112072,1.410918e+07,2023-01-20 12:02:21-05:00
1576,1.254240,5.253376,1.252168,5082.480913,2.500673,4.200124,9,4.108112,1.411766e+07,2023-01-20 14:23:43-05:00


In [173]:
# Parsing the cycle by cycle metrics

df_agg = df.groupby('cycle_index').agg('max')

# Cumulative metrics
df_by_cyc = pd.DataFrame()
df_by_cyc.index = df_agg.index

df_by_cyc['charge_capacity_ah'] = df_agg['charge_capacity_ah']
df_by_cyc['charge_energy_wh'] = df_agg['charge_energy_wh']
df_by_cyc['discharge_capacity_ah'] = df_agg['discharge_capacity_ah']
df_by_cyc['discharge_energy_wh'] = df_agg['discharge_energy_wh']
df_by_cyc['tot_charge_capacity_ah'] = df_agg['charge_capacity_ah'].cumsum()
df_by_cyc['tot_charge_energy_wh'] = df_agg['charge_energy_wh'].cumsum()
df_by_cyc['tot_discharge_capacity_ah'] = df_agg['discharge_capacity_ah'].cumsum()
df_by_cyc['tot_discharge_energy_wh'] = df_agg['discharge_energy_wh'].cumsum()


df_by_cyc['charge_capacity_ah'].loc[df_rpt['cycle_index'].unique()] = np.NaN
df_by_cyc['charge_energy_wh'].loc[df_rpt['cycle_index'].unique()] = np.NaN
df_by_cyc['discharge_capacity_ah'].loc[df_rpt['cycle_index'].unique()] = np.NaN
df_by_cyc['discharge_energy_wh'].loc[df_rpt['cycle_index'].unique()] = np.NaN




# Parse metrics by step type
CYC_STEP_INDEX_CHARGE_CC = 3
CYC_STEP_INDEX_CHARGE_CV = 4
CYC_STEP_INDEX_CHARGE_REST = 5
CYC_STEP_INDEX_DISCHARGE_CC = 6
CYC_STEP_INDEX_DISCHARGE_REST = 7

charge_cc_time_s = []

for row in df_by_cyc.iterrows():

    this_df = df[df['cycle_index'] == row[0]]
    
    
    df_chgcc = this_df[this_df['step_index'] == CYC_STEP_INDEX_CHARGE_CC]
    
    if df_chgcc.empty:
        charge_cc_time_s.append(np.NaN)
    else:
        charge_cc_time_s.append(df_chgcc['test_time_s'].tail(1).item() - df_chgcc['test_time_s'].head(1).item())

# Assign back to the DataFrame    
df_by_cyc['charge_cc_time_s'] = charge_cc_time_s





plt.figure(figsize=(10,6))
plt.plot(df_by_cyc['tot_discharge_capacity_ah'], df_by_cyc['discharge_capacity_ah'])




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [174]:
df_by_cyc

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,discharge_energy_wh,tot_charge_capacity_ah,tot_charge_energy_wh,tot_discharge_capacity_ah,tot_discharge_energy_wh,charge_cc_time_s
cycle_index,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.000000,0.000000,0.192217,0.612502,NaN
2,NaN,NaN,NaN,NaN,0.081963,0.268955,0.199162,0.632655,NaN
3,NaN,NaN,NaN,NaN,0.163932,0.550408,0.206113,0.654522,NaN
4,NaN,NaN,NaN,NaN,0.245902,0.837872,0.213059,0.677289,NaN
5,NaN,NaN,NaN,NaN,0.327873,1.126976,0.220013,0.700393,NaN
...,...,...,...,...,...,...,...,...,...
1574,1.257079,5.264991,1.254994,4.117342,2245.402472,9044.422953,2243.579642,7706.754823,401.580803
1575,1.255640,5.259053,1.253413,4.112072,2246.658112,9049.682006,2244.833056,7710.866895,398.485173
1576,1.254240,5.253376,1.252168,4.108112,2247.912352,9054.935383,2246.085223,7714.975007,396.081990


In [46]:
# Assemble a list of cycle indices where the RPT starts

rpt_start_cycle_list = []

unique_cycles = df['cycle_index'].unique()

for cycle in unique_cycles:
    
    this_df = df[df['cycle_index'] == cycle]
    
    if this_df['step_index'].isin([RPT_STEP_INDEX_START]).any():
        
        rpt_start_cycle_list.append(cycle)
        
rpt_start_cycle_list

[1, 85, 166, 297, 426, 554, 682, 809, 935, 1060, 1184, 1306, 1427, 1547]

In [85]:
len(c20_charge_voltage_v)

5411

In [107]:
fh1, axs1 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh2, axs2 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh3, axs3 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh4, axs4 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)

for rpt_start_cycle in rpt_start_cycle_list:
    
    # Create a DataFrame to hold the current RPT data only
    this_rpt = df_rpt[(df_rpt['cycle_index'] >= rpt_start_cycle)]
    this_rpt = this_rpt[this_rpt['cycle_index'] <= rpt_start_cycle + 80]
    this_rpt['time_s'] = this_rpt['test_time_s'] - this_rpt['test_time_s'].iloc[0]
    
    # Extract the charge and discharge C/20 curves
    c20_charge_capacity_ah = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_CHARGE]['charge_capacity_ah']
    c20_charge_voltage_v   = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_CHARGE]['voltage_v']
    c20_discharge_capacity_ah = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_DISCHARGE]['discharge_capacity_ah']
    c20_discharge_voltage_v   = this_rpt[this_rpt['step_index'] == RPT_STEP_INDEX_C20_DISCHARGE]['voltage_v']
    
    # Calculate the filtered dV/dQ outputs
    window_length=101
    polyorder=2
    Qf_d = signal.savgol_filter(c20_discharge_capacity_ah,window_length,polyorder)
    dQ_d = signal.savgol_filter(c20_discharge_capacity_ah,window_length,polyorder,1)
    Vf_d = signal.savgol_filter(c20_discharge_voltage_v,window_length,polyorder)
    dV_d = signal.savgol_filter(c20_discharge_voltage_v,window_length,polyorder,1)
    dVdQ_d = dV_d / dQ_d    
    dQdV_d = dQ_d / dV_d
    
    Qf_c = signal.savgol_filter(c20_charge_capacity_ah,window_length,polyorder)
    dQ_c = signal.savgol_filter(c20_charge_capacity_ah,window_length,polyorder,1)
    Vf_c = signal.savgol_filter(c20_charge_voltage_v,window_length,polyorder)
    dV_c = signal.savgol_filter(c20_charge_voltage_v,window_length,polyorder,1)
    dVdQ_c = dV_c / dQ_c    
    dQdV_c = dQ_c / dV_c
    
    # 
    
    # Make the plots
    axs1[0].plot(this_rpt['time_s'], this_rpt['voltage_v'])
    axs1[1].plot(this_rpt['time_s'], this_rpt['step_index'])

    axs2[0].plot(c20_charge_capacity_ah, c20_charge_voltage_v)
    axs2[1].plot(c20_discharge_capacity_ah, c20_discharge_voltage_v)
    
    axs3[0].plot(Qf_c, dVdQ_c)
    axs3[1].plot(Qf_d, dVdQ_d)

    axs3[0].set_ylim([0, 0.75])
    axs3[1].set_ylim([-0.75, 0])
    
    axs4[0].plot(Vf_c, dQdV_c)
    axs4[1].plot(Vf_d, dQdV_d)

    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(10, nrows * 4), sharex=True)

i = 0
for yvar, ax in zip(plot_vars, axs):
    ax.plot(df_rpt[xvar], df_rpt[yvar], ls='', marker='o', ms=1, c='r')
    ax.plot(df_cyc[xvar], df_cyc[yvar], ls='', marker='o', ms=1, c='b')
    ax.set_ylabel(yvar)
    if i < len(axs):
        ax.set_xticks([])
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
CYC_STEP_INDEX_CHARGE_CC = 3
CYC_STEP_INDEX_CHARGE_CV = 4
CYC_STEP_INDEX_CHARGE_REST = 5
CYC_STEP_INDEX_DISCHARGE_CC = 6
CYC_STEP_INDEX_DISCHARGE_REST = 7